In [137]:
import requests
import pandas as pd

In [138]:
BASE_URL = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'

#Dataset name
ArtworksFountainsMonuments = 'public-artworks-fountains-and-monuments'
MemorialsSculptures = 'public-memorials-and-sculptures'
OutdoorArtworks = 'outdoor-artworks'

NUMBER_OF_RECORDS = 10

# Add a filter for the number of results returned
FILTERS = 'records?limit={0}&offset=0&timezone=UTC'.format(NUMBER_OF_RECORDS)
# Same as
FILTERS = f'records?limit={NUMBER_OF_RECORDS}&offset=0&timezone=UTC'

In [139]:
ArtworksFountainsMonumentsURL = f'{BASE_URL}{publicArtworksFountainsMonuments}/{FILTERS}'
ArtFountainsResult = requests.get(publicArtworksFountainsMonumentsURL)

MemorialsSculpturesURL = f'{BASE_URL}{publicMemorialsSculptures}/{FILTERS}'
MemorialSculpturesResult = requests.get(publicMemorialsSculpturesURL)

OutdoorArtworksURL = f'{BASE_URL}{OutdoorArtworks}/{FILTERS}'
OutdoorArtworksResult = requests.get(OutdoorArtworksURL)

In [140]:
print(ArtFountainsResult)
print(MemorialSculpturesResult)
print(OutdoorArtworksResult)

<Response [200]>
<Response [200]>
<Response [200]>


In [141]:
# Use links & iterate calls

def get_data(base, data_url, size = 0):
    
    filters = f'records?limit={10}&offset={size}&timezone=UTC'
    url = f'{base}{data_url}/{filters}'
    result = requests.get(url)
    result_json = result.json()
    max_results = result_json['total_count']
    links = result_json['links']
    records = result_json['records']
    records_df = pd.json_normalize(records)
    
    # fix columns names
    records_df.drop(columns=['links'],inplace=True)
    column_names = records_df.columns.values.tolist()
    
    # Replace location.lat & location.lon
    column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='location' else a for a in column_names]
    column_names = [i.split('.')[-1] for i in column_names]
    records_df.columns = column_names
    
    next_url = None
    
    # get next url
    if records_df.shape[0] != max_results:
        for l in links:
            if l['rel']=='next':
                next_url = l['href']
                
    return [records_df, next_url, column_names]

In [142]:
# Create function to add API output to dataframe
def add_to_dataframe(dataframe, new_data):
    df = pd.concat([dataframe, new_data])
    return df

In [143]:
ArtworksFountainsMonumentsDF, url_next, column_names = get_data(BASE_URL,ArtworksFountainsMonuments)

In [144]:
url_next

'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/public-artworks-fountains-and-monuments/records?limit=10&offset=10&timezone=UTC&include_app_metas=False&include_links=False'

In [145]:
column_names

['id',
 'timestamp',
 'size',
 'asset_type',
 'name',
 'xorg',
 'xsource',
 'address_point',
 'artist',
 'alternate_name',
 'art_date',
 'mel_way_ref',
 'respective_author',
 'structure',
 'lon',
 'lat',
 'easting',
 'northing']

In [148]:
# Create new empty dataframe with columns
NEW_AFMDF_DF = pd.DataFrame(columns=column_names)
NEW_AFMDF_DF

,id,timestamp,size,asset_type,name,xorg,xsource,address_point,artist,alternate_name,art_date,mel_way_ref,respective_author,structure,lon,lat,easting,northing


In [149]:
# Add initial output to datframe
NEW_AFMDF_DF = add_to_dataframe(NEW_AFMDF_DF, ArtworksFountainsMonumentsDF)
NEW_AFMDF_DF.head()

,id,timestamp,size,asset_type,name,xorg,xsource,address_point,artist,alternate_name,art_date,mel_way_ref,respective_author,structure,lon,lat,easting,northing
0,84ef8d6f74366bd58a851cb9e62148669b93293f,2022-11-13T09:58:36.005Z,314,Fountain,Coles Fountain,City of Melbourne,MCC - Ortho Image March 2005 - Final,"517 Albert Street, EAST MELBOURNE",Robert Woodward,None,1981,2F_J1,City Of Melbourne,Stainless-steel fountain on bluestone paving,144.973484,-37.809771,321609.36000,5.813357e+06
1,e1b70019e59b104d764e708d5920e0ec682368cc,2022-11-13T09:58:36.005Z,351,Monument,Black Swan Memorial Drinking Fountain,City of Melbourne,MCC - Ortho Image March 2005 - Final,"Alexandra Gardens, St Kilda Road, MELBOURNE",Raymond B. Ewers,None,1974,2F_J6,City Of Melbourne,Bluestone drinking fountain with bronze plaque,144.971824,-37.819571,321486.86700,5.812266e+06
2,7b0e72c727c7b6d3085eda217982d318476fd018,2022-11-13T09:58:36.005Z,334,Art,Federation Bells,City of Melbourne,MCC - Ground Ortho Image March 2008,"Birrarung Marr, MELBOURNE",Designers Neil McLachlan and Anton Hasell,None,2002,2F_ K6,City of Melbourne,Bronze-alloy bells on galvanised-steel poles,144.974159,-37.818697,321690.25256,5.812367e+06
3,815b6aa3fc2b4f29c01c77767ba16d7fb2902652,2022-11-13T09:58:36.005Z,334,Art,Federation Bells,City of Melbourne,MCC - Ground Ortho Image March 2008,"Birrarung Marr, MELBOURNE",Designers Neil McLachlan and Anton Hasell,None,2002,2F_ K6,City of Melbourne,Bronze-alloy bells on galvanised-steel poles,144.974129,-37.818688,321687.56694,5.812368e+06
4,0c718f6d02d17e51eeb3f3b0b8d283d7bbebc23e,2022-11-13T09:58:36.005Z,283,Art,Coat of Arms,City of Melbourne,MCC - Ortho Image March 2005 - Final,"104 Swanston Street, MELBOURNE",City of Melbourne,None,1992,2F_F4,City Of Melbourne,Brass pavement inlay,144.966444,-37.815048,321002.34200,5.812757e+06


In [152]:
# Get the rest of the Dataset
while url_next != None:
    df_new, url_next, column_names = get_data(BASE_URL, ArtworksFountainsMonuments, NEW_AFMDF_DF.shape[0])
    NEW_AFMDF_DF = add_to_dataframe(NEW_AFMDF_DF, df_new)
    print(NEW_AFMDF_DF.shape)

(20, 18)
(30, 18)
(40, 18)
(50, 18)
(60, 18)
(70, 18)
(80, 18)
(90, 18)
(100, 18)
(110, 18)
(120, 18)
(130, 18)
(140, 18)
(150, 18)
(160, 18)
(170, 18)
(180, 18)
(190, 18)
(200, 18)
(210, 18)
(220, 18)
(230, 18)
(240, 18)
(250, 18)
(260, 18)
(261, 18)


In [154]:
MemorialsSculpturesDF, url_next, column_names = get_data(BASE_URL,MemorialsSculptures)

# Create new empty dataframe with columns
NEW_MS_DF = pd.DataFrame(columns=column_names)
NEW_MS_DF

,id,timestamp,size,description,title,lon,lat


In [155]:
# Get the rest of the Dataset
while url_next != None:
    df_new, url_next, column_names = get_data(BASE_URL, MemorialsSculptures, NEW_MS_DF.shape[0])
    NEW_MS_DF = add_to_dataframe(NEW_MS_DF, df_new)
    print(NEW_MS_DF.shape)

(10, 7)
(20, 7)
(30, 7)
(40, 7)
(50, 7)
(60, 7)
(70, 7)
(80, 7)
(90, 7)
(100, 7)
(110, 7)
(120, 7)
(130, 7)
(140, 7)
(150, 7)
(160, 7)
(163, 7)


In [158]:
OutdoorArtworksDF, url_next, column_names = get_data(BASE_URL, OutdoorArtworks)

# Create new empty dataframe with columns
NEW_OA_DF = pd.DataFrame(columns=column_names)
NEW_OA_DF

,id,timestamp,size,lon,lat,type,coordinates,type,road_segment,location,...,object_type,art_date,inscription,longitude,company,service_manager,makers,property,classification,title


In [159]:
# Get the rest of the Dataset
while url_next != None:
    df_new, url_next, column_names = get_data(BASE_URL, OutdoorArtworks, NEW_OA_DF.shape[0])
    NEW_OA_DF = add_to_dataframe(NEW_OA_DF, df_new)
    print(NEW_OA_DF.shape)

(10, 25)
(20, 25)
(30, 25)
(40, 25)
(50, 25)
(60, 25)
(70, 25)
(80, 25)
(90, 25)
(100, 25)
(110, 25)
(120, 25)
(130, 25)
(140, 25)
(150, 25)
(160, 25)
(170, 25)
(180, 25)
(190, 25)
(200, 25)
(202, 25)


In [160]:
import folium

m = folium.Map(location=[-37.818663392, 144.9833294], zoom_start = 16)

folium.Marker(
    location=[-37.818663392, 144.9833294],
    popup='Melbourne Cricket Ground',
    icon=folium.Icon(icon = 'flag',color='orange')
).add_to(m)

for index, row in NEW_AFMDF_DF.iterrows():
    lat = row['lat']
    lon = row['lon'] 
    name = row['name']
    
    folium.Marker([lat, lon], popup=name).add_to(m)
m

In [ ]:
import folium

m = folium.Map(location=[-37.818663392, 144.9833294], zoom_start = 16)

folium.Marker(
    location=[-37.818663392, 144.9833294],
    popup='Melbourne Cricket Ground',
    icon=folium.Icon(icon = 'flag',color='orange')
).add_to(m)

for index, row in NEW_AFMDF_DF.iterrows():
    lat = row['lat']
    lon = row['lon'] 
    name = row['name']
    
    folium.Marker([lat, lon], popup=name).add_to(m)
m

In [161]:
import folium

m = folium.Map(location=[-37.818663392, 144.9833294], zoom_start = 16)

folium.Marker(
    location=[-37.818663392, 144.9833294],
    popup='Melbourne Cricket Ground',
    icon=folium.Icon(icon = 'flag',color='orange')
).add_to(m)

for index, row in NEW_OA_DF.iterrows():
    lat = row['lat']
    lon = row['lon'] 
    name = row['name']
    
    folium.Marker([lat, lon], popup=name).add_to(m)
m

KeyError: 'name'